# NF21 - Air Pollution & Health - Data Exploration


In [3]:
import polars as pl
from pathlib import Path

## Data Extraction

### EDGAR (polluants)

In [6]:
edgar_dir = Path('data/EDGAR')
edgar_dirs = [d for d in edgar_dir.iterdir() if d.is_dir()]

poll_dfs = {}

for subdir in edgar_dirs:
    code = subdir.name.split('_')[0]
    print(f"extracting data for {code}")
    df = pl.read_excel(list(subdir.glob('*.xlsx'))[0], engine='calamine', read_options={"header_row": 9})
    poll_dfs[code.lower()] = df
    
print(poll_dfs)

extracting data for CO
extracting data for SO2
extracting data for NOx
extracting data for NMVOC
extracting data for PM25
extracting data for PM10
extracting data for NH3
extracting data for BC
extracting data for OC
{'co': shape: (4_043, 61)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ IPCC_anne ┆ C_group_I ┆ Country_c ┆ Name     ┆ … ┆ Y_2019    ┆ Y_2020    ┆ Y_2021    ┆ Y_2022    │
│ x         ┆ M24_sh    ┆ ode_A3    ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---       ┆ ---       ┆ ---       ┆ str      ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ str       ┆ str       ┆ str       ┆          ┆   ┆           ┆           ┆           ┆           │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Non-Annex ┆ Rest      ┆ ABW       ┆ Aruba    ┆ … ┆ 0.011641  ┆ 0.011268  ┆ 0.011693  ┆ 0.011693  │
│ _I        ┆ Central   ┆           ┆          ┆  

In [7]:
edgar = pl.concat(
    [df.with_columns(pl.lit(name).alias("Pollutant")) for name, df in poll_dfs.items()],
    how="vertical"
)

In [26]:
edgar.describe

<bound method DataFrame.describe of shape: (41_537, 62)
┌────────────┬────────────┬────────────┬──────────┬───┬──────────┬──────────┬──────────┬───────────┐
│ IPCC_annex ┆ C_group_IM ┆ Country_co ┆ Name     ┆ … ┆ Y_2020   ┆ Y_2021   ┆ Y_2022   ┆ Pollutant │
│ ---        ┆ 24_sh      ┆ de_A3      ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---       │
│ str        ┆ ---        ┆ ---        ┆ str      ┆   ┆ f64      ┆ f64      ┆ f64      ┆ str       │
│            ┆ str        ┆ str        ┆          ┆   ┆          ┆          ┆          ┆           │
╞════════════╪════════════╪════════════╪══════════╪═══╪══════════╪══════════╪══════════╪═══════════╡
│ Non-Annex_ ┆ Rest       ┆ ABW        ┆ Aruba    ┆ … ┆ 0.011268 ┆ 0.011693 ┆ 0.011693 ┆ co        │
│ I          ┆ Central    ┆            ┆          ┆   ┆          ┆          ┆          ┆           │
│            ┆ America    ┆            ┆          ┆   ┆          ┆          ┆          ┆           │
│ Non-Annex_ ┆ Rest       ┆ ABW    

### GBD 2023 (Diseases)

In [17]:
ihme_dir = Path('data/IHME')
gbd_files = [f for f in ihme_dir.iterdir() if f.name.endswith('.csv')]

diseases_dfs = {}

for f in gbd_files:
    code = f.name.split('-')[1].replace('.csv', '')
    print(f"extracting data for {code}")
    df = pl.read_csv(f)
    diseases_dfs[code.lower()] = df
    
for key, df in diseases_dfs.items():
    columns_to_rename = {col: col.replace('_name', '') for col in df.columns if col.endswith('_name')}
    if columns_to_rename:
        diseases_dfs[key] = df.rename(columns_to_rename)
        print(f"Renamed columns in {key}: {columns_to_rename}")

print(diseases_dfs)

extracting data for asthme_autres
extracting data for pneumoconioses_maladies_pulmonaires
extracting data for cancers
Renamed columns in asthme_autres: {'measure_name': 'measure', 'location_name': 'location', 'sex_name': 'sex', 'age_name': 'age', 'cause_name': 'cause', 'metric_name': 'metric'}
{'asthme_autres': shape: (49_776, 16)
┌────────────┬───────────┬────────────┬──────────┬───┬──────┬────────────┬────────────┬────────────┐
│ measure_id ┆ measure   ┆ location_i ┆ location ┆ … ┆ year ┆ val        ┆ upper      ┆ lower      │
│ ---        ┆ ---       ┆ d          ┆ ---      ┆   ┆ ---  ┆ ---        ┆ ---        ┆ ---        │
│ i64        ┆ str       ┆ ---        ┆ str      ┆   ┆ i64  ┆ f64        ┆ f64        ┆ f64        │
│            ┆           ┆ i64        ┆          ┆   ┆      ┆            ┆            ┆            │
╞════════════╪═══════════╪════════════╪══════════╪═══╪══════╪════════════╪════════════╪════════════╡
│ 1          ┆ Décès     ┆ 140        ┆ Bahreïn  ┆ … ┆ 1980 ┆

In [23]:
# Get the columns from the 'cancers' dataframe as reference
reference_columns = diseases_dfs['cancers'].columns

# Select only the reference columns from each dataframe
standardized_dfs = []
for name, df in diseases_dfs.items():
    # Select only columns that exist in both the current df and reference columns
    common_columns = [col for col in reference_columns if col in df.columns]
    standardized_df = df.select(common_columns)
    standardized_dfs.append(standardized_df)

gbd = pl.concat(standardized_dfs, how="vertical")

In [25]:
gbd.describe

<bound method DataFrame.describe of shape: (145_248, 10)
┌───────────┬──────────┬───────┬─────────────────┬───┬──────┬───────────┬───────────┬───────────┐
│ measure   ┆ location ┆ sex   ┆ age             ┆ … ┆ year ┆ val       ┆ upper     ┆ lower     │
│ ---       ┆ ---      ┆ ---   ┆ ---             ┆   ┆ ---  ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ str      ┆ str   ┆ str             ┆   ┆ i64  ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪══════════╪═══════╪═════════════════╪═══╪══════╪═══════════╪═══════════╪═══════════╡
│ Décès     ┆ Bahreïn  ┆ Homme ┆ Normalisé selon ┆ … ┆ 1980 ┆ 27.496092 ┆ 48.572561 ┆ 10.026657 │
│           ┆          ┆       ┆ l'âge           ┆   ┆      ┆           ┆           ┆           │
│ Décès     ┆ Bahreïn  ┆ Femme ┆ Normalisé selon ┆ … ┆ 1980 ┆ 10.445785 ┆ 20.898253 ┆ 3.128085  │
│           ┆          ┆       ┆ l'âge           ┆   ┆      ┆           ┆           ┆           │
│ Décès     ┆ Bahreïn  ┆ Homme ┆ Normalisé selon ┆ … ┆ 1980 ┆

### Écriture des fichiers

In [28]:
edgar.write_csv('data/combined/edgar.csv')
gbd.write_csv('data/combined/gbd.csv')